In [19]:
import tensorflow as tf
import keras
from keras import layers
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [2]:
train_dir = '/content/drive/MyDrive/CatClassifier/train'

In [3]:
train_datagen = ImageDataGenerator(validation_split=0.2,rescale=1./255)

In [4]:
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(224,224),
    batch_size=32,
    subset='training',
    class_mode='binary',
)

validation_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(224,224),
    batch_size=32,
    subset='validation',
    class_mode='binary'
)

Found 20001 images belonging to 2 classes.
Found 5000 images belonging to 2 classes.


In [20]:
def inception(input_tensor, filter_channels):
    filter_1x1, filter_3x3_B, filter_3x3, filter_5x5_B, filter_5x5, max_pool_A = filter_channels
    # filter_3x3_B : 3x3 컨볼루션 전(Before)의 1x1 컨볼루션의 필터 갯수
    # filter_5x5_B : 5x5 컨볼루션 전(Before)의 1x1 컨볼루션의 필터 갯수
    # max_pool_A : 3x3 Max Pooling층 이후(After)의 1x1 컨볼루션의 필터 갯수
    branch_1 = keras.layers.Conv2D(filter_1x1, (1, 1), strides=1, padding='same', activation='relu')(input_tensor)
    
    branch_2 = keras.layers.Conv2D(filter_3x3_B, (1, 1), strides=1, padding='same', activation='relu')(input_tensor)
    branch_2 = keras.layers.Conv2D(filter_3x3, (3, 3), strides=1, padding='same', activation='relu')(branch_2)

    branch_3 = keras.layers.Conv2D(filter_5x5_B, (1, 1), strides=1, padding='same', activation='relu')(input_tensor)
    branch_3 = keras.layers.Conv2D(filter_5x5, (5, 5), strides=1, padding='same', activation='relu')(branch_3)
    
    branch_4 = keras.layers.MaxPooling2D((3, 3), strides=1, padding='same')(input_tensor)
    branch_4 = keras.layers.Conv2D(max_pool_A, (1, 1), strides=1, padding='same', activation='relu')(branch_4)
    
    DepthConcat = keras.layers.Concatenate()([branch_1, branch_2, branch_3, branch_4])
    
    return DepthConcat

In [23]:
def GoogLeNet(model_input, classes=2):
    conv_1 = layers.Conv2D(64, (7, 7), strides=2, padding='same', activation='relu')(model_input)
    pool_1 = layers.MaxPooling2D((3, 3), strides=2, padding='same')(conv_1) 
    LRN_1 = layers.Lambda(tf.nn.local_response_normalization)(pool_1) # (56, 56, 64)
    
    conv_2 = layers.Conv2D(64, (1, 1), strides=1, padding='valid', activation='relu')(LRN_1) # (56, 56, 64)
    conv_3 = layers.Conv2D(192, (3, 3), strides=1, padding='same', activation='relu')(conv_2) # (56, 56, 192)
    LRN_2 = layers.Lambda(tf.nn.local_response_normalization)(conv_3) # (56, 56, 192)
    pool_2 = layers.MaxPooling2D((3, 3), strides=2, padding='same')(LRN_2) # (28, 28, 192)
    
    inception_3a = inception(pool_2, [64, 96, 128, 16, 32, 32]) # (28, 28, 256)
    inception_3b = inception(inception_3a, [128, 128, 192, 32, 96, 64]) # (28, 28, 480)
    
    pool_3 = layers.MaxPooling2D((3, 3), strides=2, padding='same')(inception_3b) # (14, 14, 480)
    
    inception_4a = inception(pool_3, [192, 96, 208, 16, 48, 64]) # (14, 14, 512)
    inception_4b = inception(inception_4a, [160, 112, 224, 24, 64, 64]) # (14, 14, 512)
    inception_4c = inception(inception_4b, [128, 128, 256, 24, 64, 64]) # (14, 14, 512)
    inception_4d = inception(inception_4c, [112, 144, 288, 32, 64, 64]) # (14, 14, 528)
    inception_4e = inception(inception_4d, [256, 160, 320, 32, 128, 128]) # (14, 14, 832)
    
    pool_4 = layers.MaxPooling2D((3, 3), strides=2, padding='same')(inception_4e) # (7, 7, 832)
    
    inception_5a = inception(pool_4, [256, 160, 320, 32, 128, 128]) # (7, 7, 832)
    inception_5b = inception(inception_5a, [384, 192, 384, 48, 128, 128]) # (7, 7, 1024)
    
    avg_pool = layers.GlobalAveragePooling2D()(inception_5b)
    dropout = layers.Dropout(0.4)(avg_pool)
    
    linear = layers.Dense(1000, activation='relu')(dropout)
    
    model_output = layers.Dense(classes, activation='softmax', name='main_classifier')(linear) # 'softmax'
    
    # Auxiliary Classifier

    # 데이터 양이 많지 않아 제거했음. 추후 tuning 등으로 추가 학습 고려 시 사용할 것.

    # auxiliary_4a = layers.AveragePooling2D((5, 5), strides=3, padding='valid')(inception_4a)
    # auxiliary_4a = layers.Conv2D(128, (1, 1), strides=1, padding='same', activation='relu')(auxiliary_4a)
    # auxiliary_4a = layers.Flatten()(auxiliary_4a)
    # auxiliary_4a = layers.Dense(1024, activation='relu')(auxiliary_4a)
    # auxiliary_4a = layers.Dropout(0.7)(auxiliary_4a)
    # auxiliary_4a = layers.Dense(classes, activation='softmax', name='auxiliary_4a')(auxiliary_4a)
    
    # auxiliary_4d = layers.AveragePooling2D((5, 5), strides=3, padding='valid')(inception_4d)
    # auxiliary_4d = layers.Conv2D(128, (1, 1), strides=1, padding='same', activation='relu')(auxiliary_4d)
    # auxiliary_4d = layers.Flatten()(auxiliary_4d)
    # auxiliary_4d = layers.Dense(1024, activation='relu')(auxiliary_4d)
    # auxiliary_4d = layers.Dropout(0.7)(auxiliary_4d)
    # auxiliary_4d = layers.Dense(classes, activation='softmax', name='auxiliary_4d')(auxiliary_4d)
    
    
    model = keras.Model(model_input, model_output)
    
    return model

In [24]:
input_shape = (224,224,3)
model_input = layers.Input(shape=input_shape)
model = GoogLeNet(model_input)

In [25]:
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_5 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
conv2d_116 (Conv2D)             (None, 112, 112, 64) 9472        input_5[0][0]                    
__________________________________________________________________________________________________
max_pooling2d_26 (MaxPooling2D) (None, 56, 56, 64)   0           conv2d_116[0][0]                 
__________________________________________________________________________________________________
lambda_4 (Lambda)               (None, 56, 56, 64)   0           max_pooling2d_26[0][0]           
____________________________________________________________________________________________

In [27]:
model.compile(loss='sparse_categorical_crossentropy',
              optimizer=keras.optimizers.SGD(lr=0.001, momentum=0.9, decay = 0.0005),
              metrics=['accuracy'])

In [28]:
hist = model.fit_generator(train_generator,
                 steps_per_epoch=100, 
                 epochs = 15 , 
                 validation_data=validation_generator ,
                 validation_steps=50 ,
                 verbose=1)

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/15
100/100 [==============================] - 2740s 27s/step - loss: 0.6932 - accuracy: 0.5001 - val_loss: 0.6931 - val_accuracy: 0.4988
Epoch 2/15
100/100 [==============================] - 2048s 21s/step - loss: 0.6928 - accuracy: 0.5304 - val_loss: 0.6933 - val_accuracy: 0.5013
Epoch 3/15
100/100 [==============================] - 1641s 16s/step - loss: 0.6935 - accuracy: 0.4976 - val_loss: 0.6931 - val_accuracy: 0.4931
Epoch 4/15
100/100 [==============================] - 1288s 13s/step - loss: 0.6932 - accuracy: 0.5068 - val_loss: 0.6930 - val_accuracy: 0.5150
Epoch 5/15
100/100 [==============================] - 1004s 10s/step - loss: 0.6931 - accuracy: 0.4955 - val_loss: 0.6931 - val_accuracy: 0.4938
Epoch 6/15
100/100 [==============================] - 843s 8s/step - loss: 0.6930 - accuracy: 0.5175 - val_loss: 0.6928 - val_accuracy: 0.5150
Epoch 7/15
100/100 [==============================] - 660s 7s/step - loss: 0.6929 - accuracy: 0.5119 - val_loss: 0.6933 - val_accura

In [29]:
model.save('/content/drive/MyDrive/CatClassifier/GoogLeNet')

INFO:tensorflow:Assets written to: /content/drive/MyDrive/CatClassifier/GoogLeNet/assets


In [30]:
model.save('/content/drive/MyDrive/CatClassifier/GoogLeNet/GoogLeNet_ver0.h5')